In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import time

In [2]:
#
html = 'https://fr.openfoodfacts.org/'
r = requests.get(html)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
#Récupération de la balise contenant les pages du sites
pages = soup.find_all('ul', attrs={'id':'pages'})

In [5]:
#Vérification de la donnée récupérée
pages[0].contents

[<li class="unavailable">Pages :</li>,
 <li class="current"><a href="">1</a></li>,
 <li><a href="/2">2</a></li>,
 <li><a href="/3">3</a></li>,
 <li><a href="/4">4</a></li>,
 <li class="unavailable">…</li>,
 <li><a href="/8059">8059</a></li>,
 <li><a href="/8060">8060</a></li>,
 <li><a href="/8061">8061</a></li>,
 <li><a href="/2" rel="next$nofollow">Suivante</a></li>,
 <li class="unavailable">(100 produits par page)</li>]

In [6]:
#Récupération des valeurs contenues dans la balise ayant comme ID "page"
records = []
for page in pages[0]:
    for child in page.descendants:
        try:
            records.append(int(str(child)))
        except ValueError:
            continue
print(records)
last_page = max(records) #récupération de la valeur maximum pour connaitre le nombre de page maximum
print(last_page)   

[1, 2, 3, 4, 8055, 8056, 8057]
8057


In [7]:
#Fonction permettant de vérifier qu'un élément n'est pas vide, sinon la valeur "XXX" est retournée
def errorAttribute(verification):
    if not verification:
        return 'XXX'
    else:
        return verification

In [8]:
# Fonction permettant d'afficher le temps de conputation des fonctions une fois excécutées
def elapsed_time(start):
    elapsed_time_fl = (time.time() - start)
    print('Temps de computation des fonctions jouées est de : {} s'.format(elapsed_time_fl))

In [9]:
# Fonction permettant de trouver le nom du produit et de le retourner
def search_product_name(soup10):
    try:
        product_name = errorAttribute(soup10.find('title').text.strip())
        return str(product_name)
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [10]:
# Fonction permettant de trouver le code barre du produit et de le retourner
def search_barre_code(soup10):
    try:
        barre_code = errorAttribute(soup10.find('p', attrs={'id':'barcode_paragraph'}).text.strip())
        if (barre_code != 'XXX'):
            barre_code_clean = re.sub("\D","", barre_code) #Seul la valeur numérique est retournée
            return str(barre_code_clean)
        else:
            return barre_code
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [11]:
# Fonction permettant de trouver le nutriscore du produit et de le retourner
def search_nutriscore(soup10):
    try:
        search_nutriscore = errorAttribute(soup10.find_all('a', attrs={'title':'Mode de calcul de la note nutritionnelle de couleur'}))
        if (search_nutriscore != 'XXX'):
            element = search_nutriscore[1].find('img', src=True)
            if not element['alt']: #Si la valeur de propriété est vide, on retourne "XXX" 
                return 'XXX'
            else: #Sinon on récupére le dernier caratère de la valeur contenu dans la propriété
                nutriscore = element['alt'][-1]
                return str(nutriscore)
        else:
            return search_nutriscore
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [12]:
# Fonction permettant de trouver le nova du produit et de le retourner
def search_nova(soup10):
    try:
        search_nova = errorAttribute(soup10.find_all('a', attrs={'title':'Classification NOVA pour la transformation des aliments'}))
        if(search_nova != 'XXX'):
            element = search_nova[1].find('img', src=True)
            if not element['alt']:  #Si la valeur de propriété est vide, on retourne "XXX" 
                return 'XXX'
            else: #Sinon on récupére le 1er caractère de la valeur contenu dans la propriété
                nova = element['alt'][0]
                return str(nova)
        else:
            return search_nova
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX' 

In [13]:
# Fonction permettant de trouver l'eco-score du produit et de le retourner
def search_ecoscore(soup10):    
    try:
        search_ecoscore = errorAttribute(soup10.find_all('a', attrs={'title':"Information sur l'Éco-score"}))
        if (search_ecoscore != 'XXX'):
            element = search_ecoscore[1].find('img', src=True)
            if not element['alt']: #Si la valeur de propriété est vide, on retourne "XXX" 
                print('XXX')
            else: #Sinon on récupére le dernier caractère de la valeur contenu dans la propriété
                ecoscore = element['alt'][-1]
                return str(ecoscore)
        else:
            return search_ecoscore
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [14]:
# Fonction permettant de trouver la dénomination du produit et de la retourner
def search_denomination(soup10):
    try:
        denomination = errorAttribute(soup10.find('span', attrs={'itemprop':'description'}).text)
        return str(denomination)
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [15]:
# Fonction permettant de trouver la quantité du produit et de la retourner
def search_quantity(soup10):
    try:
        search_quantity = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_quantity != 'XXX'):
            quantity =''
            for element in search_quantity:
                match_element = re.search('Quantité.*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    quantity = element.next_sibling #On récupère la valeur du prochain élément
            if not quantity: #Si la valeur de propriété est vide, on retourne "XXX"
                return 'XXX'
            else: #Sinon on retourne la valeur contenu dans la variable
                return str(quantity)
        else:
            return search_quantity
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
            return 'XXX'

In [16]:
# Fonction permettant de trouver le ou les conditionnements du produit et de le/les retourner
def search_conditionnement(soup10):
    try:
        search_all_conditionnement = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_conditionnement != 'XXX'):                                        
            records = []
            for element in search_all_conditionnement:
                match_element = re.search('Conditionnement*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    conditionnement_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (conditionnement_parent != 'XXX'):
                        all_conditionnements = errorAttribute(conditionnement_parent.find_all('a')) 
                        if (all_conditionnements != 'XXX'):
                            for conditionnement in all_conditionnements:
                                records.append(conditionnement.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_conditionnements
                    else:
                        return conditionnement_parent
            if not records: #Si aucune valeur "matche" (records vide), on retourne "XXX"
                return 'XXX'
        else:
            return search_all_conditionnement
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
            return 'XXX'

In [17]:
# Fonction permettant de trouver la marque du produit et de la retourner
def search_marque(soup10):
    try:
        marque = errorAttribute(soup10.find('a', attrs={'itemprop':'brand'}).text)
        return str(marque)
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [18]:
# Fonction permettant de trouver le ou les catégories du produit et de le/les retourner
def search_categories(soup10):
    try:
        search_all_categories = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_categories != 'XXX'):                                        
            records = []
            for element in search_all_categories:
                match_element = re.search('Catégories*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    categories_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (categories_parent != 'XXX'):
                        all_categories = errorAttribute(categories_parent.find_all('a'))
                        if (all_categories != 'XXX'):
                            for categorie in all_categories:
                                records.append(categorie.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_categories
                    else:
                        return categories_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_categories
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
            return 'XXX'

In [19]:
# Fonction permettant de trouver le ou les labels/certifications/recompenses du produit et de le/les retourner
def search_label_cert_rec(soup10):
    try:
        search_all_label_cert_rec = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_label_cert_rec != 'XXX'):                                        
            records = []
            for element in search_all_label_cert_rec:
                match_element = re.search('Labels, certifications, récompenses*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    label_cert_rec_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (label_cert_rec_parent != 'XXX'):
                        all_label_cert_rec = errorAttribute(label_cert_rec_parent.find_all('a'))
                        if (all_label_cert_rec != 'XXX'):
                            for label_cert_rec in all_label_cert_rec:
                                records.append(label_cert_rec.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_label_cert_rec
                    else:
                        return label_cert_rec_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_label_cert_rec
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [20]:
# Fonction permettant de trouver l'origine du produit et de la retourner
def search_origin(soup10):
    try:
        search_all_origin = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_origin != 'XXX'):                                        
            records = []
            for element in search_all_origin:
                match_element = re.search('Origine des ingrédients*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    origin_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (origin_parent != 'XXX'):
                        origin = errorAttribute(origin_parent.find('a'))
                        if (origin != 'XXX'):
                            records.append(origin.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return origin
                    else:
                        return origin_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_origin
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [21]:
# Fonction permettant de trouver le ou les lieux de fabrication ou de transformation du produit et de le/les retourner
def search_loc_transf(soup10):
    try:
        search_all_loc_transf = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_loc_transf != 'XXX'):                                        
            records = []
            for element in search_all_loc_transf:
                match_element = re.search('Lieux de fabrication ou de transformation*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    loc_transf_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (loc_transf_parent != 'XXX'):
                        all_loc_transf = errorAttribute(loc_transf_parent.find_all('a'))
                        if (all_loc_transf != 'XXX'):
                            for loc_transf in all_loc_transf:
                                records.append(loc_transf.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_loc_transf
                    else:
                        return loc_transf_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_loc_transf
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [22]:
# Fonction permettant de trouver le ou les traces du produit et de le/les retourner
def search_trace(soup10):
    try:
        search_all_trace = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_trace != 'XXX'):                                        
            records = []
            for element in search_all_trace:
                match_element = re.search('Code de traçabilité*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    trace_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (trace_parent != 'XXX'):
                        all_trace = errorAttribute(trace_parent.find_all('a'))
                        if (all_trace != 'XXX'):
                            for trace in all_trace:
                                records.append(trace.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_trace
                    else:
                        return trace_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_trace
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [23]:
# Fonction permettant de trouver le lien du fabricant du produit et de la retourner
def search_link_fab(soup10):
    try:
        search_all_link_fab = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_link_fab != 'XXX'):                                        
            records = []
            for element in search_all_link_fab:
                match_element = re.search('Lien vers la page du produit sur le site officiel du fabricant*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    link_fab_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (link_fab_parent != 'XXX'):
                        all_link_fab = errorAttribute(link_fab_parent.find_all('a'))
                        if (all_link_fab != 'XXX'):
                            for link_fab in all_link_fab:
                                records.append(link_fab.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_link_fab
                    else:
                        return link_fab_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_link_fab
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [24]:
# Fonction permettant de trouver le ou les magasin ou sont vendus le produit et de le/les retourner
def search_magasins(soup10):
    try:
        search_all_magasins = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_magasins != 'XXX'):                                        
            records = []
            for element in search_all_magasins:
                match_element = re.search('Magasins*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    magasins_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (magasins_parent != 'XXX'):
                        all_magasins = errorAttribute(magasins_parent.find_all('a'))
                        if (all_magasins != 'XXX'):
                            for magasins in all_magasins:
                                records.append(magasins.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_magasins
                    else:
                        return magasins_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_magasins
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [25]:
# Fonction permettant de trouver le ou les pays de vente du produit et de le/les retourner
def search_pays_vente(soup10):
    try:
        search_all_pays_vente = errorAttribute(soup10.find_all('span', attrs={'class':'field'}))
        if (search_all_pays_vente != 'XXX'):                                        
            records = []
            for element in search_all_pays_vente:
                match_element = re.search('Pays de vente*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    pays_vente_parent = errorAttribute(element.find_parent('p')) #On récupère la balise parent
                    if (pays_vente_parent != 'XXX'):
                        all_pays_vente = errorAttribute(pays_vente_parent.find_all('a'))
                        if (all_pays_vente != 'XXX'):
                            for pays_vente in all_pays_vente:
                                records.append(pays_vente.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return 'XXX'
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_pays_vente
                    else:
                        return pays_vente_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_pays_vente
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [26]:
# Fonction permettant de trouver les analyses des ingrédients du produit et de le/les retourner
def search_analyse_ingredients(soup10):
    try:
        search_all_analyse_ingredients = errorAttribute(soup10.find_all('span', attrs={'class':'alert round label ingredients_analysis green'}))
        if (search_all_analyse_ingredients != 'XXX'):
            records = []
            for element in search_all_analyse_ingredients:
                records.append(element.text.strip())
            if not records: #Si records vide, on retourne "XXX"
                return('XXX')
            else:
                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                return str_join
        else:
            return search_all_analyse_ingredients
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [27]:
# Fonction permettant de trouver le ou les additifs du produit et de le/les retourner
def search_additifs(soup10):
    try:
        search_all_additifs = errorAttribute(soup10.find_all('b'))
        if (search_all_additifs != 'XXX'):
            records = []
            for element in search_all_additifs:
                match_element = re.search('Additifs*', element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    additifs_parent = errorAttribute(element.find_parent('div')) #On récupère la balise parent
                    if (additifs_parent != 'XXX'):
                        all_additifs = errorAttribute(additifs_parent.find_all('a'))
                        if (all_additifs != 'XXX'):
                            for additif in all_additifs:
                                if re.search('E\d', additif.get_text()):
                                    records.append(additif.text.strip())
                            if not records: #Si records vide, on retourne "XXX"
                                return('XXX')
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_additifs
                    else:
                        return additifs_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_additifs
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'  

In [28]:
# Fonction permettant de trouver les ingrédients issus de l'huile de palme du produit et de le/les retourner
def search_ingredient_palme(soup10):
    try:
        search_all_ingredient_palme = errorAttribute(soup10.find_all('b'))
        if (search_all_ingredient_palme != 'XXX'):
            records = []
            for element in search_all_ingredient_palme:
                match_element = re.search("Ingrédients issus de l'huile de palme*", element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
                if (match_element):
                    ingredient_palme_parent = errorAttribute(element.find_parent('div')) #On récupère la balise parent
                    if (ingredient_palme_parent != 'XXX'):
                        all_ingredient_palme = errorAttribute(ingredient_palme_parent.find_all('a'))
                        if (all_ingredient_palme != 'XXX'):
                            for ingredient_palme in all_ingredient_palme:
                                records.append(ingredient_palme.text)
                            if not records: #Si records vide, on retourne "XXX"
                                return('XXX')
                            else:
                                str_join = ", ".join(records) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
                                return str_join
                        else:
                            return all_ingredient_palme
                    else:
                        return ingredient_palme_parent
            if not records: #Si records vide, on retourne "XXX"
                return 'XXX'
        else:
            return search_all_ingredient_palme
    except AttributeError: #Valeur "XXX" retournée si la balise est inexsitant
        return 'XXX'

In [29]:
# Fonction permettant de trouver les repères nutritionnelles du produit et de le/les retourner
def search_reperes_nut(soup10, result):
    increment = 1
    index_list = 0
    try:
        search_all_reperes_nut = errorAttribute(soup10.find_all('h4'))
    except AttributeError:
        search_all_reperes_nut = 'XXX'
    if (search_all_reperes_nut != 'XXX'):
        for element in search_all_reperes_nut:
            match_element = re.search("Repères nutritionnels pour 100 g*", element.get_text()) #On regarde si l'élement "matche" avec la chaine de caractère souhaité
            if (match_element):
                try:
                    reperes_nut_parent = errorAttribute(element.find_parent('div')) #On récupère la balise parent
                except AttributeError:
                    reperes_nut_parent = 'XXX'  
                if (reperes_nut_parent != 'XXX'):
                    try:
                        all_reperes_nut = errorAttribute(reperes_nut_parent.find_all('img'))
                    except AttributeError:
                        all_reperes_nut = 'XXX'
                    if (all_reperes_nut != 'XXX'):
                        while index_list < len(all_reperes_nut): #Valeur "XXX" retournée si la balise est inexsitant
                            if(all_reperes_nut[index_list]):
                                result.append(re.sub("[^0-9.]", "", all_reperes_nut[index_list].next_sibling))
                            index_list+=1
                            increment+=1
                        if (index_list < 4): #Valeur "XXX" retournée si la balise est inexsitant
                            while increment <= 4:
                                result.append('XXX')
                                increment+=1
                    else: #Valeur "XXX" retournée si la balise est inexsitant
                        while increment <= 4:
                            result.append('XXX')
                            increment+=1
                else:
                    while increment <= 4:
                        result.append('XXX')
                        increment+=1
        if (increment <= 4): #Valeur "XXX" retournée si la balise est inexsitant
            while increment <= 4:
                result.append('XXX')
                increment+=1

    else: #Valeur "XXX" retournée si la balise est inexsitant
        while increment <= 4:
            result.append('XXX')
            increment+=1

In [30]:
# Fonction permettant de trouver les informations nutritionnelles du produit et de le/les retourner
def search_nutriment_info(soup10, result):
    try:
        str_join_class =''
        search_products_comparaison_checked = errorAttribute(soup10.find('input', attrs={'checked':'checked'}))
    except AttributeError:
        search_products_comparaison_checked = 'XXX'
    if (search_products_comparaison_checked != 'XXX'):
        product_comparaison_class =''
        result.append(search_products_comparaison_checked.next_sibling.strip())
        try:
            search_products_comparaison = errorAttribute(soup10.find_all('th'))
        except AttributeError:
            pass
        if (search_products_comparaison_checked != 'XXX'):
            for element in search_products_comparaison:
                match_element = re.search(search_products_comparaison_checked.next_sibling.strip(), element.get_text())
                if (match_element):
                    product_comparaison_class = element['class']
            str_join_class = " ".join(product_comparaison_class) #Jointure effectuée pour regrouper toute les valeurs en une seul chaine de caractère
    else:
        result.append('XXX')
         
    try:
        all_val_product_nutriment_kj = errorAttribute(soup10.find('tr', attrs={'id':'nutriment_energy-kj_tr'}))
    except AttributeError:
        all_val_product_nutriment_kj = 'XXX'
    if (all_val_product_nutriment_kj != 'XXX'):
        try:
            val_product_nutriment_kj = errorAttribute(all_val_product_nutriment_kj.find('td', attrs={'class':'nutriment_value'}))
        except AttributeError:
            val_product_nutriment_kj = 'XXX'   
        if (val_product_nutriment_kj != 'XXX'):
            element_kj = re.sub("[^0-9.]", "", val_product_nutriment_kj.text)
            if not element_kj:
                result.append('XXX')
            else:
                result.append(element_kj)        
        else:
            result.append('XXX')
        try:
            val_product_nutriment_comparaison_kj = errorAttribute(all_val_product_nutriment_kj.find('td', attrs={'class':str_join_class}))
        except AttributeError:
            val_product_nutriment_comparaison_kj = 'XXX'
        if (val_product_nutriment_comparaison_kj != 'XXX'):
            try:
                val_product_kj = re.sub("[^0-9.]", "", val_product_nutriment_comparaison_kj.find('span', attrs={'class':'compare_value'}).text)
            except AttributeError:
                val_product_kj = 'XXX'
            if not val_product_kj:
                result.append('XXX')
            else:
                result.append(val_product_kj)
        else:
            result.append('XXX')
    else:
            result.append('XXX')
            result.append('XXX')
    

    try:
        all_val_product_nutriment_kc = errorAttribute(soup10.find('tr', attrs={'id':'nutriment_energy-kcal_tr'}))
    except AttributeError:
        all_val_product_nutriment_kc = 'XXX'
    if (all_val_product_nutriment_kc != 'XXX'):
        try:
            val_product_nutriment_kc = errorAttribute(all_val_product_nutriment_kc.find('td', attrs={'class':'nutriment_value'}))
        except AttributeError:
            val_product_nutriment_kc = 'XXX'   
        if (val_product_nutriment_kc != 'XXX'):
            element_kc = re.sub("[^0-9.]", "", val_product_nutriment_kj.text)
            if not element_kc:
                result.append('XXX')
            else:
                result.append(element_kc)
        else:
            result.append('XXX')
        val_product_nutriment_comparaison_kc = errorAttribute(all_val_product_nutriment_kc.find('td', attrs={'class':str_join_class}))
        if (val_product_nutriment_comparaison_kc != 'XXX'):
            try:
                val_product_kc = re.sub("[^0-9.]", "", val_product_nutriment_comparaison_kc.find('span', attrs={'class':'compare_value'}).text)
            except AttributeError:
                val_product_kc = 'XXX'
            if not val_product_kc:
                result.append('XXX')
            else:
                result.append(val_product_kc)
        else:
            result.append('XXX')
    else:
            result.append('XXX')
            result.append('XXX')


In [31]:
# Fonction permettant de trouver l'impact environnemental du produit et de le/les retourner
def search_impact_environnemental(soup10):
    return search_ecoscore(soup10)

In [32]:
# Fonction permettant de regrouper les informations de récupération de donnée et de les jouer
def my_Function2(url):
    start = time.time()
    result = []
    r = requests.get(url)
    soup10 = BeautifulSoup(r.text, 'html.parser')
    result.append(search_product_name(soup10))
    result.append(search_barre_code(soup10))
    result.append(search_nutriscore(soup10))
    result.append(search_nova(soup10))
    result.append(search_ecoscore(soup10))
    result.append(search_denomination(soup10))
    result.append(search_quantity(soup10))
    result.append(search_conditionnement(soup10))
    result.append(search_marque(soup10))
    result.append(search_categories(soup10))
    result.append(search_label_cert_rec(soup10))
    result.append(search_origin(soup10))
    result.append(search_loc_transf(soup10))
    result.append(search_trace(soup10))
    result.append(search_link_fab(soup10))
    result.append(search_magasins(soup10))
    result.append(search_pays_vente(soup10))
    result.append(search_analyse_ingredients(soup10))
    result.append(search_additifs(soup10))
    result.append(search_ingredient_palme(soup10))
    search_reperes_nut(soup10, result)
    search_nutriment_info(soup10, result)
    elapsed_time(start)
    result.append(search_impact_environnemental(soup10))
    return result

In [33]:
# Fonction permettant de parcourir tout les produits, de jouer les fonctions de récupération de données et de créer le dataframe
def my_Function(r, html, df):
    df_info = pd.DataFrame(columns=df.columns)
    result = []
    soup = BeautifulSoup(r.text, 'html.parser')
    search_products = soup.find('ul', attrs={'class':'products'})
    for all_product in search_products.find_all('li'):
        for link_product in all_product.find_all('a'): 
            url = link_product['href']
            url_creation = html+str(url)
            print(url_creation)
            result.append(my_Function2(url_creation))
    return df_info.append(pd.DataFrame(result, columns=df.columns),ignore_index = True)
    

In [34]:
current_page = 1
df_final = pd.DataFrame(columns=['Nom du Produit',
'Code-barres (EAN/EAN-13)', 
'Nutri-score',
'NOVA',
'Eco-Score', 
'Dénomination', 
'Quantité',
'Conditionnement',
'Marques',
'Catégories', 
'Labels, certifications, récompenses', 
'Origine des ingrédients', 
'Lieux de fabrication ou de transformation', 
'Code de traçabilité', 
'Lien vers la page du produit sur le site officiel du fabricant', 
'Magasins',
'Pays de vente',
'Analyse des ingrédients',
'Additifs', 
"Ingrédients issus de lhuile de palme",
'Matières grasses / Lipides en quantité élevée', 
'Acides gras saturés en quantité élevée', 
'Sucres en quantité élevée',
'Sel en faible quantité', 
'Comparaison avec les valeurs moyennes des produits de même catégorie (Catégories cochées)', 
'Energie (kj)',
'Energie produit comparé (kj)',
'Nombre de calorie (Énergie (kcal))',
'Nombre de calorie produit comparé (Énergie (kcal))', 
'Impact environnemental',])

In [35]:
df_final.columns

Index(['Nom du Produit', 'Code-barres (EAN/EAN-13)', 'Nutri-score', 'NOVA',
       'Eco-Score', 'Dénomination', 'Quantité', 'Conditionnement', 'Marques',
       'Catégories', 'Labels, certifications, récompenses',
       'Origine des ingrédients', 'Lieux de fabrication ou de transformation',
       'Code de traçabilité',
       'Lien vers la page du produit sur le site officiel du fabricant',
       'Magasins', 'Pays de vente', 'Analyse des ingrédients', 'Additifs',
       'Ingrédients issus de lhuile de palme',
       'Matières grasses / Lipides en quantité élevée',
       'Acides gras saturés en quantité élevée', 'Sucres en quantité élevée',
       'Sel en faible quantité',
       'Comparaison avec les valeurs moyennes des produits de même catégorie (Catégories cochées)',
       'Energie (kj)', 'Energie produit comparé (kj)',
       'Nombre de calorie (Énergie (kcal))',
       'Nombre de calorie produit comparé (Énergie (kcal))',
       'Impact environnemental'],
      dtype='object'

In [36]:
df_temp = df_final.copy()

In [37]:
# Fonction permettant de trouver les informations nutritionnelles du produit et de le/les retourner
df_final.to_csv('openfoodfact.csv', index=False, sep=';', encoding='utf-8')

In [38]:
# Cette boucle permet de parcourir toute les pages, de jouer la fonction principale de récupération et d'implémenter le CSV
while current_page <= last_page:
    start = time.time()
    html_link = html+str(current_page)
    print(html_link)
    r = requests.get(html_link)
    data = df_temp.append(my_Function(r, html, df_final))
    data.to_csv('openfoodfact.csv', mode = 'a', header = False, index=False, sep=';', encoding='utf-8')
    elapsed_time_fl = (time.time() - start)
    print(elapsed_time_fl)
    current_page+=1

https://fr.openfoodfacts.org/1
https://fr.openfoodfacts.org//produit/3274080005003/cristaline-eau-de-source
Temps de computation des fonctions jouées est de : 0.3444082736968994 s
https://fr.openfoodfacts.org//produit/3017620422003/nutella-ferrero
Temps de computation des fonctions jouées est de : 0.4359731674194336 s
https://fr.openfoodfacts.org//produit/7622210449283/prince-chocolat-lu
Temps de computation des fonctions jouées est de : 0.5359518527984619 s
https://fr.openfoodfacts.org//produit/5449000000996/coca-cola
Temps de computation des fonctions jouées est de : 0.4400351047515869 s
https://fr.openfoodfacts.org//produit/3017620425035/nutella-ferrero
Temps de computation des fonctions jouées est de : 0.5446128845214844 s
https://fr.openfoodfacts.org//produit/3229820129488/muesli-raisin-figue-abricot-bjorg
Temps de computation des fonctions jouées est de : 0.601215124130249 s
https://fr.openfoodfacts.org//produit/3175680011480/biscuit-sesame-gerble
Temps de computation des fonctio

Temps de computation des fonctions jouées est de : 0.6054537296295166 s
https://fr.openfoodfacts.org//produit/3033710084913/arome-maggi
Temps de computation des fonctions jouées est de : 0.3102293014526367 s
https://fr.openfoodfacts.org//produit/3251490332080/biscuits-figue-et-son-gerble
Temps de computation des fonctions jouées est de : 0.3896465301513672 s
https://fr.openfoodfacts.org//produit/3045140105502/chocolat-au-lait-du-pays-alpin-milka
Temps de computation des fonctions jouées est de : 0.41544198989868164 s
https://fr.openfoodfacts.org//produit/7613035833272/perrier
Temps de computation des fonctions jouées est de : 0.5355086326599121 s
https://fr.openfoodfacts.org//produit/8712100325953/amora-moutarde-de-dijon-fine-et-forte-bocal
Temps de computation des fonctions jouées est de : 0.3991570472717285 s
https://fr.openfoodfacts.org//produit/3168930010906/quaker-cruesli-chocolat-noir
Temps de computation des fonctions jouées est de : 0.5031886100769043 s
https://fr.openfoodfacts

Temps de computation des fonctions jouées est de : 0.4170844554901123 s
https://fr.openfoodfacts.org//produit/3175681066632/biscuits-pavot-citron-gerble
Temps de computation des fonctions jouées est de : 0.46372509002685547 s
https://fr.openfoodfacts.org//produit/3387390321357/fitness-nature-cereales-nestle
Temps de computation des fonctions jouées est de : 0.4955015182495117 s
https://fr.openfoodfacts.org//produit/3068320114453/badoit
Temps de computation des fonctions jouées est de : 0.9382040500640869 s
https://fr.openfoodfacts.org//produit/3061990141354/bn-chocolat
Temps de computation des fonctions jouées est de : 0.4374070167541504 s
https://fr.openfoodfacts.org//produit/3155251205500/beurre-doux-gastronomique-president
Temps de computation des fonctions jouées est de : 0.40004539489746094 s
https://fr.openfoodfacts.org//produit/7622210476296/the-biscuits-lu-the
Temps de computation des fonctions jouées est de : 0.4127473831176758 s
https://fr.openfoodfacts.org//produit/303349000

KeyboardInterrupt: 

In [ ]:
df_test = pd.read_csv("openfoodfact.csv", sep=';', encoding='utf-8')

In [ ]:
df_test